In [ ]:
import torch
from diffusers import DiffusionPipeline
from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
from PIL import Image, ImageOps
import os
import warnings

warnings.filterwarnings("ignore")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
    'tencent/Hunyuan3D-2.1',
    torch_dtype=torch.float16
)

image_gen_pipeline = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")


def resize_and_pad(image: Image.Image, target_size: tuple[int, int], fill_color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    padded_image = Image.new("RGB", target_size, fill_color)
    
    image.thumbnail(target_size, Image.Resampling.LANCZOS)
    
    paste_position = (
        (target_size[0] - image.width) // 2,
        (target_size[1] - image.height) // 2
    )
    
    padded_image.paste(image, paste_position)
    
    return padded_image


def generate_mesh_from_image(image_path: str, output_path: str):
    if not os.path.exists(image_path):
        print(f"Ошибка: Файл изображения не найден: {image_path}")
        return

    image = Image.open(image_path).convert("RGB")
    
    processed_image = resize_and_pad(image, (512, 512))

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    ref_image_path = os.path.splitext(output_path)[0] + '.png'
    processed_image.save(ref_image_path)
    print(f"Референсное изображение сохранено: {ref_image_path}")

    generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=False)[0]
    generated_mesh.export(output_path)
    
    print(f"Меш из изображения сохранен: {output_path}")


def generate_mesh_from_text(prompt: str, output_path: str):
    image = image_gen_pipeline(prompt=prompt).images[0]
    
    processed_image = resize_and_pad(image, (512, 512))

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    ref_image_path = os.path.splitext(output_path)[0] + '.png'
    processed_image.save(ref_image_path)
    print(f"Референсное изображение сохранено: {ref_image_path}")

    generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=False)[0]
    generated_mesh.export(output_path)
    
    print(f"Меш из текста '{prompt}' сохранен: {output_path}")


2025-08-02 20:55:14,808 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1
2025-08-02 20:55:14,815 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1/model.fp16.ckpt


using moe
using moe
using moe
using moe
using moe
using moe
PointCrossAttentionEncoder INFO: pc_sharpedge_size is zero


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
generate_mesh_from_image(
    image_path='../assets/d98mtp939hpc1.jpeg',
    output_path='../output/rebecca_one_piece_bikini.glb'
)


Референсное изображение сохранено: ../output/rebecca_one_piece_bikini.png


Volume Decoding: 100%|██████████████████████████████████| 7134/7134 [00:13<00:00, 513.18it/s]


Меш из изображения сохранен: ../output/rebecca_one_piece_bikini.glb


In [8]:
generate_mesh_from_text(
    prompt="(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus",
    output_path='../output/sword.glb'
)

  0%|          | 0/50 [00:00<?, ?it/s]

Референсное изображение сохранено: ../output/sword.png


Volume Decoding: 100%|██████████████████████████████████| 7134/7134 [00:13<00:00, 514.27it/s]


Меш из текста '(masterpiece, best quality, 8k), (one single sword:1.5), a single ornate fantasy sword, solo object, centered, full body shot of a sword, detailed weapon, game asset, ((isolated on a pure white background)), flat lighting, no shadows' сохранен: ../output/sword.glb
